In [0]:
import yaml
with open('./ingestion_config.yaml', 'r') as file:
    config = yaml.safe_load(file)

databricks_config = config["databricks_config"]
volume_path = databricks_config["volume_path"]

catalog = config["databricks_config"]["catalog"]
metadata_schema = config["databricks_config"]["metadata_schema"]
schema_subset = config["databricks_config"]["db_subset"].split(',')

metadata_path = f"{volume_path}dev_20240627/dev_tables.json"
sql_lite_fqdn = f"{volume_path}dev_20240627/dev_databases/dev_databases/"

In [0]:
if schema_subset:
  dbs = schema_subset
else:
  db_records = spark.table(f"{catalog}.{metadata_schema}.table_metadata").select("db_id").distinct().collect()
  dbs = [db["db_id"] for db in db_records]

dbs

In [0]:
from pyspark.sql.functions import lit

for db in dbs:
    for t in dbutils.fs.ls(f"{sql_lite_fqdn}{db}/database_description_tsv"):
        table_name = t.name.replace(".tsv", "")
        db_description = (
            spark.read.option("header", True)
            .option("inferSchema", True)
            .option("delimiter", "\t")
            .csv(t.path)
            .withColumn("table_name", lit(table_name))
        )
        db_description.write.mode("append").saveAsTable(f"{catalog}.{db}.table_metadata")